In [2]:
from pyvi.pyvi import ViTokenizer,ViPosTagger

def readFile_txt(filename):
    data=[]
    f = open(filename)
#     next(f)
    for row in f:
        data.append(row)
    return data

def make_data_train(data):
    data_train=[]
    for row in data:
        post_tup=ViPosTagger.postagging(ViTokenizer.tokenize(row))
        post_list=tupToList(post_tup)
        data_train.append(post_list)
    return data_train

def writeListFile(filename,data):
    str_out=''
    for item in data:
        for tup in item:
            str_out+=tup[0]+" "+tup[1]+" "+tup[2]+"\n"
        str_out+="\n"
#     print(str_out)
    F = open(filename,"w")
    F.write(str(str_out))
def tupToList(tup):
    list1=[]
    for item in tup[0]:
        index=tup[0].index(item)
        tup_=(item,tup[1][index],'0')
        list1.append(tup_)
    return list1

In [3]:
# tạo file tiền huấn luyện
data=readFile_txt("train1")
data_train=make_data_train(data)
# print(len(data_train))
# print(data_train)
writeListFile("train1.pre",data_train)
# tạo file test tiền huấn luyện
data=readFile_txt("test1")
data_test=make_data_train(data)
# print(len(data_test))
# print(data_test)
writeListFile("test1.pre",data_test)

In [4]:
tup1=ViPosTagger.postagging(ViTokenizer.tokenize(u"Trường đại học Bách Khoa Hà Nội"))
# tup1[0]
# tup1[1]
tupToList(tup1)

[('Trường', 'N', '0'),
 ('đại_học', 'N', '0'),
 ('Bách_Khoa', 'Np', '0'),
 ('Hà_Nội', 'Np', '0')]

In [5]:
# create data train
data_train=readFile_txt("train1.pre")
data_test=readFile_txt("test1.pre")
def getMatrix(data):
    matrix=[]
    vector=[]
    line=0
    for dt in data:
        if len(dt)>1:
            dt = dt[:-1]
            l1=dt.split(" ")
            t1=(l1[0],l1[1],l1[2])
            vector.append(t1)
        else:
            matrix.append(vector)
            vector=[]
    print (len(matrix))
    return matrix
matrix_train=getMatrix(data_train)
matrix_test=getMatrix(data_test)

175
25


In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
sent2features(matrix_train[0])[0]

{'+1:postag': 'N',
 '+1:postag[:2]': 'N',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'giá',
 'BOS': True,
 'bias': 1.0,
 'postag': 'C',
 'postag[:2]': 'C',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'còn',
 'word[-2:]': 'òn',
 'word[-3:]': 'Còn'}

In [8]:
%%time
X_train = [sent2features(s) for s in matrix_train]
y_train = [sent2labels(s) for s in matrix_train]

X_test = [sent2features(s) for s in matrix_test]
y_test = [sent2labels(s) for s in matrix_test]

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 21.8 ms


In [9]:
%%time
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
import pickle
f = open('../model/ner.pkl', 'wb')
pickle.dump(crf, f)
f.close()

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 55 ms


In [115]:
labels = list(crf.classes_)
labels.remove('0')
labels

['TYP', 'STR', 'CTY', 'WRD', 'PRV', 'PRC', 'PRJ', 'SQR']

In [116]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.5952380952380952

In [117]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

        SQR      0.000     0.000     0.000         0
        PRC      0.000     0.000     0.000         1
        WRD      0.000     0.000     0.000         0
        PRJ      1.000     0.500     0.667         2
        PRV      0.000     0.000     0.000         2
        STR      1.000     0.500     0.667         2
        CTY      1.000     1.000     1.000         3
        TYP      1.000     0.500     0.667         4

avg / total      0.786     0.500     0.595        14



/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
